<a href="https://colab.research.google.com/github/gulayoklan/Ceng463-Assignment2/blob/main/task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install huggingface_hub

In [ ]:
import numpy as np
import pandas as pd
import time
import datetime
import gc
import random
from nltk.corpus import stopwords
import re

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import transformers
from transformers import BertForSequenceClassification, AdamW, BertConfig,BertTokenizer,get_linear_schedule_with_warmup, AutoTokenizer
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from accelerate import Accelerator

# Data Preprocessing

In [ ]:
file_path = "power-it-train.tsv"

# Load the dataset
df = pd.read_csv(file_path, sep='\t')

# Keep only the necessary columns (text and label) and drop rows with missing values
df = df[['text_en', 'text' ,'label']].dropna()

# Ensure the label column is binary and of type integer
df['label'] = df['label'].astype(int)

In [ ]:

# Stratified split into train, validation, and test sets
train, temp = train_test_split(df, test_size=0.1, stratify=df['label'], random_state=42)
val, test = train_test_split(temp, test_size=0.5, stratify=temp['label'], random_state=42)

test.head()
print(df['label'].unique())
print("Train label counts:\n", train['label'].value_counts())
print("Val   label counts:\n", val['label'].value_counts())
print("Test  label counts:\n", test['label'].value_counts())

In [ ]:
#oversampling to resolve the class imbalance problem
from sklearn.utils import resample

# Separate majority and minority classes
train_majority = train[train.label == 0]
train_minority = train[train.label == 1]

print("Before oversampling:")
print(train['label'].value_counts())

# Upsample minority class
train_minority_upsampled = resample(
    train_minority,
    replace=True,             # sample with replacement
    n_samples=len(train_majority),    # to match majority class
    random_state=42           # reproducible results
)

# Combine majority class with upsampled minority class
train_oversampled = pd.concat([train_majority, train_minority_upsampled])

print("After oversampling:")
print(train_oversampled['label'].value_counts())
# Shuffle the oversampled training data
train = train_oversampled.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model =BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased',num_labels=2)

In [ ]:
def preprocess(data,tokenizer):
  input_ids=[]
  attention_masks=[]
  sentences=data.text.values
  labels=data.label.values
  for sentence in sentences:
    encoded_dict=tokenizer.encode_plus(sentence,
                                            add_special_tokens=True,
                                            max_length=512,
                                            padding="max_length",
                                            truncation=True,
                                            return_attention_mask=True,
                                            return_tensors='pt')
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])
  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  labels = torch.tensor(labels)
  return TensorDataset(input_ids,attention_masks,labels)


In [ ]:
train_data=preprocess(train,tokenizer)
val_data=preprocess(val,tokenizer)
test_data=preprocess(test,tokenizer)

train_dataloader=DataLoader(train_data,batch_size=8)
validation_dataloader=DataLoader(val_data,batch_size=8)
test_dataloader=DataLoader(test_data,batch_size=8)



In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8
                )

In [ ]:

epochs = 6

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()
best_eval_accuracy = 0
# For each epoch...
for epoch_i in range(0, epochs):

    #Training
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    # Measure how long the training epoch takes.
    t0 = time.time()
    total_train_loss = 0
    model.train()
    for step, batch in enumerate(train_dataloader):
        # Unpack this training batch from our dataloader.
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        output = model(b_input_ids,
                             token_type_ids=None,
                             attention_mask=b_input_mask,
                             labels=b_labels)
        loss = output.loss
        total_train_loss += loss.item()
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        # Update parameters and take a step using the computed gradient.
        optimizer.step()
        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
    # Validation
    # After the completion of each training epoch, measure our performance on
    # our validation set.
    print("")
    print("Running Validation...")
    t0 = time.time()
    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        with torch.no_grad():
            output= model(b_input_ids,
                                   token_type_ids=None,
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
        loss = output.loss
        total_eval_loss += loss.item()
        logits = output.logits
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        total_eval_accuracy += flat_accuracy(logits, label_ids)
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    validation_time = format_time(time.time() - t0)
    if avg_val_accuracy > best_eval_accuracy:
        torch.save(model, 'bert_model_power')
        best_eval_accuracy = avg_val_accuracy
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )
print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 6 ========
Training...

  Average training loss: 0.61
  Training epoch took: 0:14:46

Running Validation...
  Accuracy: 0.72

======== Epoch 2 / 6 ========
Training...


In [ ]:
model = torch.load('bert_model_power')

<ipython-input-15-e374b6cafc0c>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('bert_model_power')


In [ ]:
predictions = []
for batch in test_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        with torch.no_grad():
            output= model(b_input_ids,
                                   token_type_ids=None,
                                   attention_mask=b_input_mask)
            logits = output.logits
            logits = logits.detach().cpu().numpy()
            pred_flat = np.argmax(logits, axis=1).flatten()

            predictions.extend(list(pred_flat))

labels=[label for label in test.label.values]
print(classification_report(labels,predictions))
print(np.unique(predictions, return_counts=True))

              precision    recall  f1-score   support

           0       0.77      0.90      0.83       491
           1       0.76      0.55      0.64       294

    accuracy                           0.77       785
   macro avg       0.76      0.72      0.73       785
weighted avg       0.77      0.77      0.76       785

(array([0, 1]), array([572, 213]))
